In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import pydicom
import cv2
import os
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import _pickle as pickle

def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

In [ ]:
imSize = 128

def find_bound(im, tres, axis):
    inf = 0
    sup = im.shape[axis]
    cond = True
    tres = tres
    temp = 0
    for i,elt in enumerate(im.sum(axis = axis)):
        if abs(elt) <=tres and cond == True:
            temp = i
        if abs(elt) >=tres:
            cond = False
            inf = temp
            

        if abs(elt)  <= tres and cond == False:
            sup = i
            cond = True
    return inf, sup

def preprocess_image(image_path, desired_size=imSize):
#     print(image_path)
    im = pydicom.dcmread(image_path).pixel_array
    
#     im = (im - im.min())/(im.max()-im.min())
    tres = 5
    inf_x, sup_x = find_bound(im, tres, 1)
    inf_y, sup_y = find_bound(im,tres,0)
    im = im[inf_x:sup_x,inf_y: sup_y]
    im = cv2.resize(im, (imSize,imSize))
    return im

def build_3d_image(ids, res = 256, to_keep = 'ALL'):
    path = './train/' + ids

    list_dir = [int(elt.split('.')[0]) for elt in os.listdir(path)]
    list_dir.sort()
    
    if to_keep == 'ALL':
        n_image_keep = len(list_dir)
    else:
        n_image_keep = to_keep
    X = list(np.zeros(n_image_keep))  #list(np.zeros(len(list_dir)))

    n_img = len(list_dir)
    print(n_img)

    for i in range(n_image_keep):
        elt = list_dir[int(i*n_img/(n_image_keep))]
        im = preprocess_image(path + '/'+ str(elt) + '.dcm', desired_size = res)
        X[i] = im
#         print(int(i*n_img/(n_image_keep)))

    X = np.array(X).astype(float)
    
    X = X.transpose(1, 2, 0)
    # X = cv2.resize(X, (imSize,imSize))
    
    X = (X - X.min())/(X.max()-X.min())
    
    return X

def make_3d_vid(X):
    %matplotlib notebook
    fig1 = plt.figure(num='Lung', figsize = (5,5))

    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel('x label')

    end = X.shape[0]
    for i in range(end):
        ax1.cla()  # Clear only 2nd figure's axes, figure 1 is ADDITIVE
        ax1.set_title('Axes title')  # Reset as removed by cla()

        ax1.imshow(X[i,:,:], cmap = 'gray')
    #     ax2.plot(range(i,end), range(i,end), 'rx')
        fig1.canvas.draw()
    #     plt.pause(0.001)

    %matplotlib notebook
    fig1 = plt.figure(num='Lung', figsize = (5,5))

    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel('x label')

    end = X.shape[1]
    for i in range(end):
        ax1.cla()  # Clear only 2nd figure's axes, figure 1 is ADDITIVE
        ax1.set_title('Axes title')  # Reset as removed by cla()

        ax1.imshow(X[:,i,:], cmap = 'gray')
    #     ax2.plot(range(i,end), range(i,end), 'rx')
        fig1.canvas.draw()
    #     plt.pause(0.001)

    %matplotlib notebook
    fig1 = plt.figure(num='Lung', figsize = (5,5))

    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel('x label')

    end = X.shape[2]
    for i in range(end):
        ax1.cla()  # Clear only 2nd figure's axes, figure 1 is ADDITIVE
        ax1.set_title('Axes title')  # Reset as removed by cla()

        ax1.imshow(X[:,:,i], cmap = 'gray')
    #     ax2.plot(range(i,end), range(i,end), 'rx')
        fig1.canvas.draw()
    #     plt.pause(0.001)
    
def metric(true, pred):
    return np.mean(abs(true-pred))

def comp_metric(true, pred, conf):
    conf1 = np.maximum(conf, np.zeros(len(conf))+70)
    
    ab = np.minimum(abs(true-pred), np.zeros(len(conf))+1000)
    
    first_term = -np.sqrt(2)*ab/conf1
    second_term = -np.log(np.sqrt(2)*conf1)
    return np.mean(first_term + second_term)

def test(model, X_train, X_test, test=True):
    print('FVC0 testing')
    model.fit(X_train, fvc0_train)
    
    if test:
        pred = model.predict(X_test)
        true = fvc0_test
    else:
        pred = model.predict(X_train)
        true = fvc0_train
        
    print(metric(true, pred))
    plt.figure(0)
    plt.scatter(true, pred)
    
    print('\n')
    print('FVC1 testing')
    model.fit(X_train, fvc1_train)
    
    if test:
        pred = model.predict(X_test)
        true = fvc1_test
    else:
        pred = model.predict(X_train)
        true = fvc1_train
        
    print(metric(true, pred))
    plt.figure(1)
    plt.scatter(true, pred)
    
    std = np.zeros(len(true))+200
    print(comp_metric(true, pred, std))
    
    print('\n')
    print('STD testing')
    model.fit(X_train, std_train)
    
    if test:
        pred = model.predict(X_test)
        true = std_test
    else:
        pred = model.predict(X_train)
        true = std_train
        
    print(metric(true, pred))
    plt.figure(2)
    plt.scatter(true, pred)
    
    print('evaluating metric with std prediction')
    model.fit(X_train, fvc1_train)
    
    pred_train = model.predict(X_train)
    pred = model.predict(X_test)
    fin = np.abs(pred_train)
    
    model.fit(X_train, fin)
    std = model.predict(X_test)
    
    print(comp_metric(fvc1_test, pred, std))
    

In [ ]:
ids = 'ID00014637202177757139317'
# ind = 4
# ids = os.listdir('./train')[ind]
# ids = 'ID00232637202260377586117'
image_path = './train/'+str(ids)+'/5.dcm'
im = pydicom.dcmread(image_path).pixel_array
# X = build_3d_image(ids,res = imSize, to_keep = 32)

# make_3d_vid(X)

In [ ]:
plt.imshow(im)

In [ ]:
im = preprocess_image(image_path, desired_size=128)


In [ ]:
im

In [ ]:
X.min()

In [ ]:
# ids = list(np.zeros(len(os.listdir('./train'))))
# data = list(np.zeros(len(os.listdir('./train'))))

ids = []
data = []

missed = 0
for i,elt in enumerate(tqdm(os.listdir('./train'))):
    try:
#         ids[i] = elt
#         data[i] = build_3d_image(elt,res = 256, to_keep = 30)
        x= build_3d_image(elt,res = 128, to_keep = 32).astype(float)
        data.append(x)
        print(x.shape)
        ids.append(elt)
        print(elt)
    except:
#         ids[i] = elt
#         data[i]= 'missing'
        print('miss')
        missed+=1
        
df = pd.read_csv('train.csv')
df = df.sort_values(by = ['Weeks'])
df.head()

FVC0 = []
FVC1 = []
STD = []
for elt in ids:
    df1 = df[df['Patient'] == elt]
    m = 100
    ind = 0
    for i, elt1 in enumerate(df1['Weeks']):
        if abs(elt1)<m:
            m = abs(elt1)
            ind = i
    FVC0.append(df1.iloc[ind]['FVC'])
    FVC1.append(df1.iloc[-1]['FVC'])
    STD.append(df1['FVC'].values.std())

In [ ]:
data = np.array(data)
ids = np.array(ids)
FVC0 = np.array(FVC0).astype(float)
FVC1 = np.array(FVC1).astype(float)
STD = np.array(STD).astype(float)


save((ids, data, FVC0, FVC1, STD), 'dataset_64')


## unet trial

In [ ]:
from unet3d import *
import tensorflow.keras as keras
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.layers import GlobalAveragePooling3D, Dense, Concatenate, Input

import tensorflow as tf
def dice_coef_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

    return 1 - (numerator + 1) / (denominator + 1)

In [ ]:
(ids, data, FVC0, FVC1, STD) = load('dataset')

In [ ]:
FVC0 = (FVC0-FVC0.mean())/1000
FVC1 = (FVC1-FVC1.mean())/1000
STD = (STD-STD.mean())/100

In [ ]:
# make_3d_vid(data[0])

In [ ]:
data = data.reshape((data.shape[0],1 , data.shape[1],data.shape[2],data.shape[3]))
X_train, X_test, fvc0_train, fvc0_test = train_test_split(data, FVC0, test_size=0.1, random_state=42)
fvc1_train, fvc1_test, std_train, std_test = train_test_split(FVC1, STD, test_size=0.1, random_state=42)

X_train = [X_train, fvc0_train]
X_test = [X_test, fvc0_test]

y_train = [fvc0_train, fvc1_train, std_train]
y_test = [fvc0_test, fvc1_test, std_test]

data = data.reshape((data.shape[0], data.shape[2],data.shape[3],data.shape[4]))

In [ ]:
input_channels, input_rows, input_cols, input_deps = 1, 128, 128, 32
num_class = 1
weight_dir = 'checkpoints/Genesis_Chest_CT.h5'
# weight_dir = './checkpoints/finetuneunet_128.h5'
models_genesis = unet_model_3d((input_channels, input_rows, input_cols, input_deps), batch_normalization=True)
print("Load pre-trained Models Genesis weights from {}".format(weight_dir))
models_genesis.load_weights(weight_dir)
x = models_genesis.get_layer('depth_13_relu').output
final_convolution = Conv3D(num_class, (1, 1, 1), activation = 'linear')(x)
output = final_convolution
# output = keras.layers.Softmax(axis=1)(final_convolution)
model = keras.models.Model(inputs=models_genesis.input, outputs=output)

optimizer = SGD(0.1)
loss = ['mae']
metrics=['mse', 'mae']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.0001, patience=6, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)
reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                                                     mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)

batch_size = 2
epochs = 90

history = model.fit(X_train, X_train, batch_size=batch_size,
    validation_data=(X_test, X_test),  epochs=epochs, callbacks = [stop, reduce])

In [ ]:
models_genesis.save_weights('./checkpoints/finetuneunet_128.h5')

In [ ]:
pred = model.predict(X_test[:2]).reshape(2,128,128,32)
true = X_test.reshape(18,128,128,32)

In [ ]:
ind = 1
make_3d_vid(true[ind])

In [ ]:
make_3d_vid(pred[ind])

In [ ]:
model.summary()

In [ ]:

# prepare the 3D model

from unet3d import *
input_channels, input_rows, input_cols, input_deps = 1, 128, 128, 32
num_class, activate = 2, 'linear'
# weight_dir = 'checkpoints/Genesis_Chest_CT.h5'
weight_dir = 'checkpoints/finetuneunet_128.h5'
models_genesis = unet_model_3d((input_channels, input_rows, input_cols, input_deps), batch_normalization=True)
print("Load pre-trained Models Genesis weights from {}".format(weight_dir))
models_genesis.load_weights(weight_dir)
x = models_genesis.get_layer('depth_7_relu').output
x = GlobalAveragePooling3D()(x)
x = Dense(1024, activation='relu')(x)

in2 = Input(shape = (1,))

x = Concatenate(axis = -1)([x, in2])

output1 = Dense(1, activation=activate)(x)
output2 = Dense(1, activation=activate)(x)
output3 = Dense(1, activation=activate)(x)
outputs = [output1, output2, output3]
model = keras.models.Model(inputs=[models_genesis.input, in2], outputs=outputs)

optimizer = SGD(0.1)
loss = ['mae', 'mae', 'mae']
metrics=['mse']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.0001, patience=6, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)
reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                                                     mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)
# batch_size = 8 #64
batch_size = 4 #128
epochs = 90

history = model.fit(X_train, y_train, batch_size=batch_size,
    validation_data=(X_test, y_test),  epochs=epochs, callbacks = [stop, reduce])

In [ ]:
model.save_weights('./checkpoints/finetuneunet_128_refined.h5')

## DEEP LEARNING

In [ ]:
from tensorflow.keras.layers import Conv3DTranspose, Activation,Conv2DTranspose, BatchNormalization, Flatten, LeakyReLU, MaxPool3D,Conv2D, Conv3D, AveragePooling3D, Reshape, Input, Dense, Add
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD,Adam
import tensorflow.keras.backend as K

In [ ]:
(ids, data, FVC0, FVC1, STD) = load('dataset')



In [ ]:
# FVC0 = np.array(FVC0).astype(float)/1000
# FVC1 = np.array(FVC1).astype(float)/1000
# STD = np.array(STD).astype(float)/100

In [ ]:
ids.shape

In [ ]:
data.shape

In [ ]:
df = pd.read_csv('train.csv')
df = df.sort_values(by = ['Weeks'])
df.head()
additionnals = []
for elt in ids:
    df1 = df[df['Patient'] == elt]
    
    vect = []
    vect.append(df1['FVC'].iloc[0])
    vect.append(df1['Percent'].iloc[0])
    vect.append(df1['Age'].iloc[0])
    
    if df1['Age'].iloc[0] == 'Male':
        vect.extend([1,0])
    else:
        vect.extend([0,1])
    
    if df1['SmokingStatus'].iloc[0] == 'Never smoked':
        vect.extend([1,0,0])
    elif df1['SmokingStatus'].iloc[0] == 'Ex-smoker':
        vect.extend([0,1,0])
    else:
        vect.extend([0,0,1])
    
    additionnals.append(vect)

additionnals = np.array(additionnals)

In [ ]:
def build_encoder(input_shape, latent_dim = 512):
    inputs = Input(shape = input_shape)
    
    x = Conv2D(32, (7,7), strides = 1, padding = 'same')(inputs)
    
    filters = [32,32,64,64,128]
    for f in filters:
        x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = BatchNormalization(axis=-1)(x)
    volumeSize = K.int_shape(x)   
    x = Flatten()(x)
    
    outputs = Dense(latent_dim)(x)
    
    model = Model(inputs, outputs)
    return model, volumeSize



In [ ]:
imSize = 128
input_shape = (imSize,imSize,1)
encoder, shape = build_encoder(input_shape, 2048)

In [ ]:
encoder.summary()

In [ ]:
encoder.load_weights('./checkpoints/encoder/encoder')

In [ ]:
X = []

for i in tqdm(range(len(data))):
    x = encoder.predict(data[i:i+1,:,:,:].transpose((3,1,2,0)))
    
    a = np.concatenate(x, axis= -1)
    
    X.append(a)
X = np.array(X)

In [ ]:
from unet3d import *
input_channels, input_rows, input_cols, input_deps = 1, 128, 128, 32
num_class, activate = 2, 'linear'
# weight_dir = 'checkpoints/Genesis_Chest_CT.h5'
weight_dir = 'checkpoints/finetuneunet_128.h5'
models_genesis = unet_model_3d((input_channels, input_rows, input_cols, input_deps), batch_normalization=True)
print("Load pre-trained Models Genesis weights from {}".format(weight_dir))
models_genesis.load_weights(weight_dir)
x = models_genesis.get_layer('depth_7_relu').output
x = GlobalAveragePooling3D()(x)
x = Dense(1024, activation='relu')(x)

in2 = Input(shape = (1,))

x = Concatenate(axis = -1)([x, in2])

output1 = Dense(1, activation=activate)(x)
output2 = Dense(1, activation=activate)(x)
output3 = Dense(1, activation=activate)(x)
outputs = [output1, output2, output3]
model = keras.models.Model(inputs=[models_genesis.input, in2], outputs=outputs)

model.load_weights('./checkpoints/finetuneunet_128_refined.h5')

In [ ]:
model.summary()

In [ ]:
# x = model.get_layer('concatenate_27').output
model1 = keras.models.Model(inputs=model.input, outputs=model.get_layer('concatenate_27').output)

X = model1.predict([data.reshape((data.shape[0],1 , data.shape[1],data.shape[2],data.shape[3])), FVC0], batch_size=3)

In [ ]:
data.shape

In [ ]:
np.array(X).shape

In [ ]:
X.shape

In [ ]:
# data = data.reshape((data.shape[0], data.shape[1],data.shape[2],data.shape[3],1))
X_train, X_test, fvc0_train, fvc0_test = train_test_split(X, FVC0, test_size=0.1, random_state=42)
fvc1_train, fvc1_test, std_train, std_test = train_test_split(FVC1, STD, test_size=0.1, random_state=42)
add_train, add_test, _ , _ = train_test_split(additionnals, additionnals, test_size=0.1, random_state=42)



y_train = [fvc0_train, fvc1_train, std_train]
y_test = [fvc0_test, fvc1_test, std_test]

del data
import gc
gc.collect()

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

# import xgboost as xgb
# clf = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                 max_depth = 5, alpha = 10, n_estimators = 10)
# import lightgbm
# clf = lightgbm.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, 
#                        n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, 
#                        min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, 
#                        subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, 
#                        reg_lambda=0.0, random_state=None, n_jobs=- 1, silent=True, 
#                        importance_type='split')

# from sklearn.neighbors import KNeighborsRegressor

# clf = KNeighborsRegressor(n_neighbors=5)

# from sklearn.ensemble import RandomForestRegressor
# clf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, 
#                             min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
#                             max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
#                             bootstrap=True, oob_score=False, n_jobs=8)

# from sklearn.svm import SVR

# clf = SVR(kernel = 'linear')

In [ ]:
test(clf,X_train, X_test, test = True)

In [ ]:
test(clf,add_train, add_test, test = True)

In [ ]:
test(clf,np.concatenate([X_train, add_train], axis = -1), np.concatenate([X_test, add_test], axis = -1), test = True)

In [ ]:
import umap
reducer = umap.UMAP(n_neighbors=3, metric='cosine', n_components=2, target_metric = 'l2')

y_test_unsupervised = np.zeros(fvc1_test.shape[0])-1
y_learn = np.concatenate([fvc1_train, y_test_unsupervised])

embedding = reducer.fit(np.concatenate([X_train, X_test], axis=0), y_learn)

Xt = embedding.transform(X_train)
Xv = embedding.transform(X_test)

In [ ]:
test(clf,Xt, Xv, test = True)

In [ ]:
test(clf,np.concatenate([Xt, add_train], axis = -1), np.concatenate([Xv, add_test], axis = -1), test = True)

In [ ]:
np.concatenate([Xt, add_train], axis = -1), np.concatenate([Xv, add_test], axis = -1)

In [ ]:
clf.fit(add_train, fvc1_train)
pred = clf.predict(add_test)
true = fvc1_test

# clf.fit(Xt, fvc1_train)
# pred = clf.predict(Xv)
# true = fvc1_test

# clf.fit(np.concatenate([Xt, add_train], axis = -1), fvc1_train)
# pred = clf.predict(np.concatenate([Xv, add_test], axis = -1))
# true = fvc1_test

In [ ]:
x = []
y = []

for i in range(80,500):
    x.append(comp_metric(true, pred, np.zeros(len(true))+i))
    y.append(i)

In [ ]:
%matplotlib notebook
plt.plot(y,x)